In [1]:
#export
import numpy as np
import torch
import pandas as pd
from pathlib import Path
import imageio
from skimage import io, transform
from torchvision import transforms

# Step 1 - Creating your dataset

The first step towards creating a dataloader is to create a dataset so that you pass in an index to get an item in the desired form.  

To understand the basic principles behind the dataset class, we will start with the most basic components.  Our simple dataset class takes in a subscriptable list for both the inputs (x) and outputs (y) and produces an tuple of the output (x, y)


In [2]:
class SimpleDataset():
    def __init__(self, x, y):
        self.x = x
        self.y = y
        assert len(x) == len(y), 'Size mismatched between inputs and labels'
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return x[idx], y[idx]

In [3]:
x = [0,1,2,3,4,5,6,7,8,9]
y = [5,3,2,6,7,8,3,4,5,6]

In [4]:
ds_train_simple = SimpleDataset(x, y)

In [5]:
print ("First element:", ds_train_simple[0])
print("Length:", len(ds_train_simple))

First element: (0, 5)
Length: 10


For our application, we want our dataset to output a tuple containing two elements:  
1. The inputs, which will be a tuple of the image and the tabular data
2. The labels

Before entering the model for training, all inputs need to be converted into torch tensors.  We have chosen to do it now, but it could also have been done at a later stage.  Similarly, image transformations can happen at any time before they are stacked together and are fed into the model.  We will perform our transformations here for simplicity.

Note: the Dataset class has been written such that elements are accessed one at a time.  This is by design as it allows you more flexibility during the collate phase; however, it is possible to restructure the __getitem__ method to process multiple indices.  The major change would be to iterate through the filenames, then open and process each individual file before stacking them together.

In [8]:
#export
class Dataset():
    DROP_WARNING = 0.9
    
    def __init__(self, df_path, img_col, cont_cols, cat_cols, target_col, image_path, suffix = '.jpg', transforms = None):
        self.df_path = df_path
        self.img_col, self.cont_cols, self.cat_cols = img_col, cont_cols, cat_cols
        self.target_col = target_col
        self.suffix = suffix
        self.image_path = Path(image_path)
        self.transforms = transforms
        
        #read in the dataframe
        self.df = pd.read_csv(df_path)
        self.df = self.clean_dataframe(self.df)

    def __len__(self):
        return len(self.df)
    
    def clean_dataframe(self, df):
        orig_len = len(df)
        
        #Remove filenames for files that do not exist (or have errors in the filepath)
        existing_files = ((pd.Series([self.image_path]*len(df))/(df[self.img_col]+'.jpg'))
                          .apply(lambda x: self.check_path_valid(x)))
        
        df.drop(df[~existing_files].index, axis = 0, inplace = True)
        
        #Remove missing values from your target columns
        df.drop(df[df[self.target_col].isna()].index, axis = 0, inplace = True)

        df.reset_index(drop=True, inplace = True)
        if len(df)/orig_len < self.DROP_WARNING: 
            print (f"Warning, more than {(1-self.DROP_WARNING)*100}% of your data was invalid")
        return df 
        
    def check_path_valid(self, path):
        try: return path.exists()
        except: return False
        
    def __getitem__(self, idx):
        filename = self.df.loc[idx, self.img_col]
        cat_data = self.df.loc[idx, self.cat_cols]
        cont_data = self.df.loc[idx, self.cont_cols].values.astype(np.float32)
        target = self.df.loc[idx, self.target_col]
        
        tabular_data = torch.tensor(cont_data)
        target = torch.tensor(target)
        
        image = io.imread(self.image_path/(filename + self.suffix))
        if self.transforms: image = self.transforms(image)
            
        return (image, tabular_data), target        

In [9]:
df_path = r'data/processed_dataframe.csv'
img_col = 'filename'
cont_cols = ['followers', 'following', 'engagement_factor_std', 'month', 'year', 'day_name', 'hour']
cat_cols = []
target_col = 'engagement_factor_moving_avg'
image_path = Path(r'data/Images')

ds_train = Dataset(df_path, 
                   img_col = img_col,
                   cont_cols = cont_cols, 
                   cat_cols = cat_cols, 
                   target_col = target_col, 
                   image_path = image_path, 
                   transforms = None)

In [10]:
print("Length:", len(ds_train), '\n')

print('First component:\n', ds_train[0][0][0], ds_train[0][0][0].shape, '\n')
print('Second component:\n', ds_train[0][0][1], '\n')
print('Third component:\n', ds_train[0][1], '\n')

Length: 13775 

First component:
 [[[237 236 242]
  [237 236 242]
  [237 236 242]
  ...
  [205 223 233]
  [213 231 241]
  [200 218 228]]

 [[232 231 237]
  [233 232 238]
  [233 232 238]
  ...
  [203 221 231]
  [203 221 231]
  [206 224 234]]

 [[230 229 235]
  [231 230 236]
  [232 231 237]
  ...
  [208 226 236]
  [203 221 231]
  [195 213 223]]

 ...

 [[226 224 237]
  [224 222 235]
  [223 221 234]
  ...
  [186 182 196]
  [184 180 194]
  [185 181 195]]

 [[226 224 237]
  [224 222 235]
  [223 221 234]
  ...
  [188 184 198]
  [186 182 196]
  [188 184 198]]

 [[226 224 237]
  [224 222 235]
  [223 221 234]
  ...
  [189 185 199]
  [188 184 198]
  [191 187 201]]] (1347, 1080, 3) 

Second component:
 tensor([1.2927e+05, 1.3410e+03, 5.1537e-01, 1.2000e+01, 2.0190e+03, 7.0000e+00,
        2.2000e+01]) 

Third component:
 tensor(1.4480) 



In our previous example, we did not include any transforms.  This will be an issue moving forward for a few reasons:
1. We did not convert our image into a torch tensor
2. There will be size mismatches across images that will make it difficult to stack
3. Pytorch expects the images to have dimension (c, h, w), but our image arrays are arranged as (h, w, c)

We can create a Transforms class, which will call a series of transforms in sequence.  The most basic transforms required to solve the above issues are a Resize transform (in which we'll rely on skimage to perform the resize), and a ToTorch() transform, which will rearrange the channels and convert the array to a torch tensor 

In [11]:
#export
class Transforms():
    def __init__(self, transforms):
        self.transforms = transforms
        
    def __call__(self, x):
        for tsfm in self.transforms:
            x = tsfm(x)
        return x
    
    
class Resize():
    def __init__(self, size):
        self.size = size
        
    def __call__(self, img):
        return transform.resize(img, (self.size, self.size))
    
    
class ToTorch():
    def __init__(self):
        pass
    
    def __call__(self, img):
        return torch.tensor(img.transpose(2,0,1))

In [12]:
df_path = r'data/processed_dataframe.csv'
img_col = 'filename'
cont_cols = ['followers', 'following', 'engagement_factor_std', 'month', 'year', 'day_name', 'hour']
cat_cols = []
target_col = 'engagement_factor_moving_avg'
image_path = Path(r'data/Images')
tfms = Transforms([Resize(255), ToTorch()])

ds_train = Dataset(df_path, 
                   img_col = img_col,
                   cont_cols = cont_cols, 
                   cat_cols = cat_cols, 
                   target_col = target_col, 
                   image_path = image_path, 
                   transforms = tfms)

In [13]:
print("Length:", len(ds_train), '\n')

print('First component:\n', ds_train[0][0][0], ds_train[0][0][0].shape, '\n')
print('Second component:\n', ds_train[0][0][1], '\n')
print('Third component:\n', ds_train[0][1], '\n')

Length: 13775 

First component:
 tensor([[[0.9098, 0.9059, 0.9020,  ..., 0.7947, 0.7990, 0.8006],
         [0.9098, 0.9059, 0.9100,  ..., 0.8171, 0.8020, 0.8015],
         [0.9098, 0.9122, 0.9140,  ..., 0.7987, 0.8016, 0.7966],
         ...,
         [0.8588, 0.8598, 0.8667,  ..., 0.8003, 0.7765, 0.7503],
         [0.8641, 0.8689, 0.8706,  ..., 0.8003, 0.7756, 0.7427],
         [0.8745, 0.8745, 0.8745,  ..., 0.7998, 0.7788, 0.7397]],

        [[0.9059, 0.9020, 0.8980,  ..., 0.8613, 0.8662, 0.8711],
         [0.9059, 0.9020, 0.9060,  ..., 0.8838, 0.8687, 0.8721],
         [0.9059, 0.9082, 0.9126,  ..., 0.8682, 0.8683, 0.8672],
         ...,
         [0.8510, 0.8520, 0.8588,  ..., 0.7847, 0.7557, 0.7268],
         [0.8563, 0.8611, 0.8627,  ..., 0.7847, 0.7557, 0.7192],
         [0.8667, 0.8667, 0.8667,  ..., 0.7842, 0.7591, 0.7201]],

        [[0.9294, 0.9255, 0.9216,  ..., 0.9006, 0.9054, 0.9104],
         [0.9294, 0.9255, 0.9296,  ..., 0.9230, 0.9079, 0.9113],
         [0.9294, 0.9318

Now we can see that our image is a tensor.  As a side effect of the resize, the values have now been scaled to the range 0-1

# Step 2 - Data Sampler

Now that we have a method of accessing individual elements from our dataset, we now need a method of arranging them into minibatches for training.  This will be accomplished by choosing the indices for each batch.  The simplest method of doing this is choosing a batch size, then dividing the data into blocks of that size (with the last block containing whatever is left over)

In [14]:
class EasySampler():
    def __init__(self, dataset, bs):
        self.dataset = dataset
        self.bs = bs #batch_size
        
        self.n = len(dataset)
        
    def __iter__(self):
        for i in range((self.n-1)//self.bs + 1):
            yield self.dataset[i*self.bs:(i+1)*self.bs]

In [15]:
simple_sampler = EasySampler([0,3,4,6,4,5,65,4, 8, 22], bs = 3)

In [16]:
for i in simple_sampler:
    print(i)
    

[0, 3, 4]
[6, 4, 5]
[65, 4, 8]
[22]


This approach works, but has a significant drawback.  Each of the minibatchs will have the same elements across each epoch of training.  Since loss is pooled across the entire minibatch (and sometimes more than one!), this limits the range of inputs it is effectively learning from.  Instead, it would be better to shuffle the data before grouping them into minibatches.  For perspective, for a dataset with 16 unique elements and a batch size of 3, you will have 4 unique minibatch combinations.  When shuffling, this number increases to 560 unique minibatch combinations.

There are many ways to shuffle the data, but a simple approach is to create an array of all the possible indices of our dataset, shuffle them and then group the shuffled indices as we did with the Easy Sampler

In [17]:
#export
class Sampler():
    def __init__(self, dataset, bs, shuffle = True):
        self.dataset = dataset
        self.bs = bs
        self.shuffle = shuffle
        
        self.n = len(self.dataset)
        
    def __iter__(self):
        #idxs = torch.randperm(self.n) if self.shuffle else torch.arange(self.n)
        idxs = np.random.permutation(self.n) if self.shuffle else np.arange(self.n)
        for i in range(0, self.n, self.bs):
            yield idxs[i:i+self.bs]
        
        
        

In [32]:
sampler = Sampler(ds_train, bs = 8)

In [37]:
for i, idxs in enumerate(sampler):
    print(f'Batch {i}: {idxs}')
    if i>5: break

Batch 0: [   55  8790  6216  9953   205  8816 11309 11210]
Batch 1: [ 5478 12825 10602  1251 13568 10500   777  6385]
Batch 2: [ 2245 13113  5805  2662  4619  7439 13329  6526]
Batch 3: [ 5068  2217  6017  8544 10194  5581  3033 12668]
Batch 4: [ 2319   606  9099 11460  8365 11501 12583  9090]
Batch 5: [ 5875 10296  9852  3097  2053 12459   989 13186]
Batch 6: [ 4332  1797 12769  7896  6597  1991  5509 12536]


We now have a sampler that provides random arrangements of our data.  Run the above cell a few times to confirm that the numbers change each time.

# Part 3 - Collate our items

The last thing we need to add is a method of assembling the individual items from each minibatch into the x and y batches we use to feed to the model.  Here, we want an output that is of the form xb, yb.  That means we will pack together both the image and tabular data into one variable.  However, we also want an easy method of splitting up the image and tabular stacks apart inside the model.  We can accomplish this by creating individual stacks of each data type, then creating two tuples: one of the tabular and image stacks (the xs tuple), the other of the xs tuple and the ys 

In [20]:
#export
def collate(data, transforms = None):
    xs, y = zip(*data)
    x1, x2 = zip(*xs)
    
    if transforms: x1 = transforms(x1)
    
    return (torch.stack(x1), torch.stack(x2)), torch.stack(y)

In [21]:
bs = 16
minibatch_samples = [ds_train[x] for x in range(bs)]
minibatch = collate(minibatch_samples)

In [22]:
xs, ys = minibatch
x1, x2 = xs

print(f"Based on a minibatch of size {bs}:", '\n')

print (f"The shape of x1 is: {x1.shape}")
print (f"The shape of x2 is: {x2.shape}")
print (f"The shape of y  is: {ys.shape}")

Based on a minibatch of size 16: 

The shape of x1 is: torch.Size([16, 3, 255, 255])
The shape of x2 is: torch.Size([16, 7])
The shape of y  is: torch.Size([16])


There you have it.  We now have a method of arranging our data into minibatches for direct input into the training cycle

# Step 4 - DataLoader (Putting it all together)

We now have all of the components required to create our dataloader.  At this point, it's really just about putting all the pieces together.  We will input our dataset, sample and collate function into the DataLoader class, then create an iterator that goes through the minibatch indices output by the sampler.  From those, we can then access the relevant components from the dataset and assemble them into the minibatch using the collate function.

In [23]:
#export
class DataLoader():
    def __init__(self, dataset, sampler, collate_func):
        self.dataset = dataset
        self.sampler = sampler
        self.collate_func = collate_func
        
    def __iter__(self):
        for idxs in self.sampler: 
            minibatch = [self.dataset[idx] for idx in idxs]
            yield (self.collate_func(minibatch))
        
        

In [24]:
dl_train = DataLoader(dataset = ds_train,
                      sampler = Sampler(ds_train, bs = 16),
                      collate_func = collate)

In [25]:
for i, (xb,yb) in enumerate(dl_train):
    print (f"Minibatch {i}, with target shape {yb.shape}")
    if i>5: break

Minibatch 0, with target shape torch.Size([16])
Minibatch 1, with target shape torch.Size([16])
Minibatch 2, with target shape torch.Size([16])
Minibatch 3, with target shape torch.Size([16])
Minibatch 4, with target shape torch.Size([16])
Minibatch 5, with target shape torch.Size([16])
Minibatch 6, with target shape torch.Size([16])


# Final Notes: 

There were a couple of things that we left out of this process.  Notable was the omission of categorical variables and how to process them.  That requires a more sophisticated look into the model itself, but for now we won't worry about that.  If you try and use categorical variables with the current implementation, it will fail because strings cannot be converted into tensors directly.  For this to work, you have to assign each category a unique number and then convert the categorical variables into their numerical equivalent  

# Script Exports

In [1]:
!python scripts/notebook2script.py "Part 1 - Dataloader.ipynb" 'dataloader.py'

Converted Part 1 - Dataloader.ipynb to scripts\dataloader.py
